**Recuerde no agregar o quitar celdas en este notebook, ni modificar su tipo. Si lo hace, el sistema automaticamente lo calificará con cero punto cero (0.0)**

Para resolver los problemas presentados, use el siguiente conjunto de datos. Use [gradetool](gradetool.md) para verificar las respuestas del notebook.

In [3]:
%load_ext bigdata
%timeout 500
%hive_init

The bigdata extension is already loaded. To reload it, use:
  %reload_ext bigdata
500
Hive initialized!


In [1]:
%%writefile tbl0.csv
1,D,4,2016-06-25,a:e:c:d,bb#10:dd#20:cc#40
2,C,4,2015-05-14,a:c,dd#40:bb#20:cc#10
3,D,6,2014-12-26,b:d,aa#10:bb#40
4,D,5,2016-06-25,a:c:e:b:d,bb#40:dd#20:aa#10:cc#30
5,C,7,2016-05-23,d:e:c,cc#10:aa#20
6,A,2,2018-06-14,a:d,aa#20
7,B,3,2014-12-22,a:e:d,cc#20
8,C,6,2015-08-20,d:a:c:e,cc#10:aa#20
9,B,3,2017-12-08,b:a:c:e,cc#40:dd#10:aa#30:bb#20
10,B,7,2015-07-28,c:d:e:a:b,aa#10:dd#40:cc#30

Writing tbl0.csv


In [ ]:
%%hive
DROP TABLE IF EXISTS tbl0;
CREATE TABLE tbl0 (c1 INT, c2 STRING, c3 INT, c4 DATE, c5 ARRAY<CHAR(1)>, c6 MAP<STRING, INT>) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' COLLECTION ITEMS TERMINATED BY ':' MAP KEYS TERMINATED BY '#' LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;

SELECT * FROM tbl0;

In [2]:
%%writefile tbl1.csv
1,7,bb,A#0:B#4:C#1:D#3:E#5
2,13,Cc,A#4:B#1:C#0:D#5:E#2
3,11,dD,A#5:B#4:C#3:D#1:E#0
4,3,BB,A#2:B#4:C#3:D#5:E#1
5,14,cc,A#4:B#0:C#2:D#5:E#3
6,8,cC,A#4:B#5:C#1:D#2:E#3
7,2,DD,A#0:B#5:C#2:D#4:E#3
8,6,dd,A#4:B#2:C#5:D#3:E#0
9,9,aa,A#1:B#4:C#2:D#5:E#3
10,4,Bb,A#2:B#3:C#4:D#1:E#0

Writing tbl1.csv


In [ ]:
%%hive
DROP TABLE IF EXISTS tbl1;
CREATE TABLE tbl1 (c1 INT, c2 INT, c3 STRING, c4 MAP<STRING, INT>) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' COLLECTION ITEMS TERMINATED BY ':' MAP KEYS TERMINATED BY '#' LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'tbl1.csv' INTO TABLE tbl1;

SELECT * FROM tbl1;

# Problema 1

Escriba una consulta que retorne los valores únicos de la columna `t0.c5` (ordenados). 

In [ ]:
%%hive
select distinct letters from (select explode(c5) as letters from tbl0) only_letters;
-- select distinct letters from tbl0 lateral view explode(c5) t0 as letters;

# Problema 2

Realice una consulta que compute la cantidad de veces que aparece cada valor de la columna `t0.c5`  por año.

In [ ]:
%%hive
SELECT date_format(c4, 'yyyy') AS year, letters, count(*) FROM tbl0 LATERAL VIEW explode(c5) t0 AS letters GROUP BY date_format(c4, 'yyyy'), letters;

# Problema 3

Escriba una consulta que retorne los elementos de la columna t0.c5 en mayúsculas, tal como se muestra a continuación.

Original:

    ["a","e","c","d"]
    ["a","c"]
    ["b","d"]
    ["a","c","e","b","d"]
    ["d","e","c"]
    ["a","d"]
    ["a","e","d"]
    ["d","a","c","e"]
    ["b","a","c","e"]
    ["c","d","e","a","b"]

Transformada:

    ["A","E","C","D"]
    ["A","C"]
    ["B","D"]
    ["A","C","E","B","D"]
    ["D","E","C"]
    ["A","D"]
    ["A","E","D"]
    ["D","A","C","E"]
    ["B","A","C","E"]
    ["C","D","E","A","B"]

In [ ]:
%%hive
SELECT COLLECT_LIST(UPPER(exploded_array)) FROM tbl0 LATERAL VIEW explode(c5) exploded_table AS exploded_array GROUP BY c1

# Problema 4

Escriba una consulta que retorne para cada valor único de la columna `t0.c2`, los valores correspondientes de la columna `
t0.c1`. 

In [ ]:
%%hive
SELECT tbl0.c2, COLLECT_SET(tbl0.c1) FROM tbl0 GROUP BY tbl0.c2;

# Problema 5

Escriba una consulta que para cada valor único de la columna `t0.c2,` calcule la suma de todos los valores asociados a las claves en la columna `t0.c6`.

In [ ]:
%%hive
SELECT c2, sum(value) FROM tbl0 LATERAL VIEW explode(c6) tbl0 GROUP BY c2;

# Problema 6

Escriba una consulta que retorne la columna `tbl0.c1` y el valor correspondiente de la columna `tbl1.c4` para la columna `tbl0.c2`.

In [ ]:
%%hive
SELECT tbl0.c1, tbl0.c2, tbl1.c4[ tbl0.c2 ] FROM tbl0 INNER JOIN tbl1 ON tbl0.c1 = tbl1.c1;

---

In [ ]:
!rm tbl* 